## PNAD Contínua Trimestral

Nesta aula prática, vamos trabalhar com dados da PNAD Contínua trimestral, utilizando códigos em R para aprender os seguintes tópicos:


1.   Leitura de microdados utilizando a biblioteca PNADcIBGE
2.   Recompor o plano amostral da pesquisa nas análises
3.   Elaborar gráficos e tabelas



📚 Inicialmente vamos conectar a sessão ao seu Google drive. Então, antes de começar a trabalhar crie uma pasta no seu drive (PNADC) e que irá salvar todos os arquivos que você baixar.


In [ ]:
#Conectando a sessão ao seu google drive 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Dessa vez, vamos trabalhar com códigos em R. Para fazer isso vamos instalar a extensão rpy2 IPython ao notebook. 
%load_ext rpy2.ipython

📌 A partir de agora, sempre que você colocar o cógido %%R no início de cada célula, você conseguirá executar comandos do R no notebook.

Utilizando a linguagem em Python, precisávamos instalar bibliotecas específicas para conseguir executar determinas comandos. No R a ideia é mesma: vamos instalar os pacotes necessários e depois temos que carrega-los toda vez que uma sessão for iniciada.

📖 Utilizaremos ao longo desta aula a biblioteca PNADcIBGE criada especificamente para leitura dos microdados da PNAD Contínua. Maiores informações sobre ela você encontra [aqui](https://cran.r-project.org/web/packages/PNADcIBGE/PNADcIBGE.pdf).

In [ ]:
#Instalando pacote da biblioteca do IBGE
%%R
install.packages("PNADcIBGE")


In [ ]:
#Carregando
%%R
library(PNADcIBGE)

In [ ]:
#Agora vamos baixar os microdados da PNAD C Trimestral referentes ao 2o trimestre de 2021
#Observe que estamos baixando os microdados em uma pasta do drive (no exemplo, uma pasta chamada PNADC)
%%R
get_pnadc(2021, quarter = 2, labels=TRUE, deflator=TRUE, design=TRUE, savedir="/content/drive/MyDrive/PNADC")

Ao final desse procedimento em sua pasta deverá conter 5 arquivos:
1.   Dicionario_e_input.zip
2.   PNADC_022021.zip
3.   dicionario_PNADC_microdados_trimestral.xls
4.   input_PNADC_trimestral.sas
5.   input_PNADC_trimestral.txt



In [ ]:
#Leitura dos microdados do 2o trimestre de 2021 da PNAD C:
%%R
PNAD022021 <- read_pnadc("/content/drive/MyDrive/PNADC/PNADC_022021.txt", "/content/drive/MyDrive/PNADC/input_PNADC_trimestral.txt", vars = NULL)

In [ ]:
#Visualizando o banco de dados
%%R
print(PNAD022021)

Variáveis que vamos trabalhar:

1.   UF: Unidade da Federação (ver no sigaa qual a sua UF)
2.   UPA: Unidade Primária de Amostragem (UPA)
3.   ESTRATO: Agrupamentos de seleção
4.   VD4001: Condição em relação à força de trabalho na semana de referência para pessoas de 14 anos ou mais de idade
5.   VD4002: Condição de ocupação na semana de referência para pessoas de 14 anos ou mais de idade
6.   V2009: Idade do morador na data de referência
7.   V2007: Sexo
8.   VD4016: Rendimento mensal habitual do trabalho principal para pessoas de 14 anos ou mais de idade (apenas para pessoas que receberam em dinheiro, produtos ou mercadorias no trabalho principal)
9.   V1028: Peso trimestral com correção de não entrevista com pós estratificação pela projeção de população



In [ ]:
#Criando um objeto contendo apenas essas variáveis
%%R
reduz <- c("UF", "UPA","Estrato", "VD4001", "VD4002", "V2009", "V2007", "VD4016","V1028")
reduz_pnad <- PNAD022021[reduz]

In [ ]:
#Conferindo se deu certo
%%R
print(reduz_pnad)

In [ ]:
#Obtendo a lista de variáveis contidas no banco de dados
%%R
names(reduz_pnad)

In [ ]:
#Retirar a notação científica
%%R 
options(scipen=999)

In [ ]:
#Para salvar os resultados com números separados por vírgula
%%R 
options(OutDec=",")


# Análise exploratória dos dados

No python trabalhamos com o tipo de variáveis, e vimos que muitas vezes precisamos transformar as variáveis de numérica pra string, string de numéricas... No R faremos o mesmo, porém por meio da função str. 

In [ ]:
#Identificando o tipo das variáveis do banco de dados
%%R
str(reduz_pnad)

In [ ]:
#Para trabalhar com as variáveis precisaremos transformar alguns de character(string)em numéricas
%%R
reduz_pnad$UF=as.numeric(reduz_pnad$UF)
reduz_pnad$UPA=as.numeric(reduz_pnad$UPA)
reduz_pnad$Estrato=as.numeric(reduz_pnad$Estrato)
reduz_pnad$VD4001=as.numeric(reduz_pnad$VD4001)
reduz_pnad$VD4002=as.numeric(reduz_pnad$VD4002)
reduz_pnad$V2007=as.numeric(reduz_pnad$V2007)


In [ ]:
#Variável condição de atividade (VD4001)
%%R
table(reduz_pnad$VD4001)


In [ ]:
#Adicionando os rótulos criando uma nova variável (condat)
%%R
reduz_pnad$condat<-factor(reduz_pnad$VD4001,labels=c('PEA', 'Inativos'))

100*prop.table(table(reduz_pnad$condat))

In [ ]:
#Variável condição de ocupação (VD4002)
%%R
table(reduz_pnad$VD4002)

In [ ]:
#Adicionando os rótulos criando uma nova variável (condocup)
%%R
reduz_pnad$condocup<-factor(reduz_pnad$VD4002,labels=c('Ocupadas', 'Desocupadas'))

100*prop.table(table(reduz_pnad$condocup))

In [ ]:
#Variável idade (V2009)
%%R
summary(reduz_pnad$V2009)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   0,00   18,00   36,00   37,11   54,00  110,00 


In [ ]:
#Construindo faixas etárias (IDADECAT)
%%R
reduz_pnad$IDADECAT <- factor(cut(reduz_pnad$V2009, breaks=c(14,20,25,30,35,40,45,50,55,60,65,110), labels=c("14-19","20-24","25-29","30-34","35-39","40-44","45-49", "50-54","55-59", "60-64","65+"),right=FALSE))

table(reduz_pnad$IDADECAT)

100*prop.table(table(reduz_pnad$IDADECAT))

In [ ]:
%%R
reduz_pnad$IDADECAT2 <- factor(cut(reduz_pnad$V2009, breaks=c(14,18,110), labels=c("14-17","18+"),right=FALSE))

table(reduz_pnad$IDADECAT2)

100*prop.table(table(reduz_pnad$IDADECAT2))

In [ ]:
#Variável sexo (V2007)
%%R
table(reduz_pnad$V2007)


     1      2 
171580 184659 


In [ ]:
#Adicionando os rótulos criando uma nova variável (sexo)
%%R
reduz_pnad$sexo<-factor(reduz_pnad$V2007,labels=c('Homens', 'Mulheres'))

100*prop.table(table(reduz_pnad$sexo))


  Homens Mulheres 
48,16429 51,83571 


In [ ]:
#Variável renda (VD4016)
%%R
summary(reduz_pnad$VD4016)


## Recompor o plano amostral da PNAD C nas análises

Uma vez que a PNAD C apresenta um desenho complexo de amostragem, ao fazer uma análise estatística desses dados é necessário incorporar a estrutura de seu plano amostral. Caso contrário, incorreções nas análises pontuais e intervalares podem ocorrer.

Para ler mais sobre análises de pesquisas com desenho amostral complexo, [clique aqui](https://www.scielo.br/pdf/rbepid/v11s1/03.pdf).


In [ ]:
#Você precisa instalar o pacote survey
%%R
install.packages("survey")

In [ ]:
#E carrega-lo
%%R
library(survey)

In [ ]:
#Recompondo o plano amostral para Brasil 
%%R
sample.pnadc <- svydesign(ids = ~UPA, strata = ~Estrato, weights = ~V1028, data = reduz_pnad , na.rm=TRUE, nest = TRUE)

# para evitar erro "has only one PSU at stage 1"
options(survey.lonely.psu = "adjust")

In [ ]:
#Agora você pode proceder às análises
#a)  Condição de atividade - Brasil
%%R
svymean(~factor(condat), design=sample.pnadc, na.rm=TRUE)

                          mean     SE
factor(condat)PEA      0,57711 0,0017
factor(condat)Inativos 0,42289 0,0017


In [ ]:
#Representação em gráfico
%%R
a<-svymean(~factor(condat), design=sample.pnadc, na.rm=TRUE)
barplot(a*100, names.arg=c("PEA","Inativos"), col="purple", 
        ylim=c(0,100), main="Condição de atividade, Brasil, 2o tri 2021.",
        sub="Fonte: PNAD C, Brasil, 2o tri 2021.")

In [ ]:
#b)  Condição de ocupação  - Brasil
%%R
svymean(~factor(condocup), design=sample.pnadc, na.rm=TRUE)

In [ ]:
#Representação em gráfico
%%R
b<-svymean(~factor(condocup), design=sample.pnadc, na.rm=TRUE)
barplot(b*100, names.arg=c("Ocupadas","Desocupadas"), col="grey", 
        ylim=c(0,100), main="Condição de ocupação, Brasil, 2o tri 2021.",
        sub="Fonte: PNAD C, Brasil, 2o tri 2021.")

In [ ]:
#c)  Renda média por sexo  - Brasil
%%R
svyby(~VD4016, ~sexo, sample.pnadc, svymean, na.rm=TRUE)

In [ ]:
# Exportando a tabela que geramos para o excel (ela ficará salva na sua sessão)
%%R
c=svyby(~VD4016, ~sexo, sample.pnadc, svymean, na.rm=TRUE)
write.table(c,file="renda_sexo.xls", sep = "\t", dec = ",")

In [ ]:
#e)  Renda média por idade  - Brasil
%%R
svyby(~VD4016, ~IDADECAT, sample.pnadc, svymean, na.rm=TRUE)

In [ ]:
# Exportando a tabela que geramos para o excel (ela ficará salva na sua sessão)
%%R
d=svyby(~VD4016, ~IDADECAT, sample.pnadc, svymean, na.rm=TRUE)
write.table(d,file="renda_idade.xls", sep = "\t", dec = ",")

In [ ]:
#Salvando a base de dados em seu drive
%%R
save(reduz_pnad,file="/content/drive/MyDrive/PNADC/reduz_pnad.Rdata")

In [ ]:
#Abrindo a base
%%R
load("/content/drive/MyDrive/PNADC/reduz_pnad.Rdata")

# Para o seu relatório, você deverá:


*   Verificar a sua UF (vide arquivo anexado ao SIGAA)
*   Criar um objeto apenas com dados da sua UF
*   Recompor o plano amostral para a sua UF
*   Criar gráficos/tabelas para a sua UF das letras a) até e) (as informaçoes das análises descritavas servem apenas de apoio, não precisa incluir as tabelas no relatório)
*   Realizar uma análise comparativa do BR com sua UF


In [ ]:
#Para criar um objeto apenas com dados da sua UF
%%R
AP <-reduz_pnad[reduz_pnad$UF==16,]

In [ ]:
%%R
print(AP)

In [ ]:
#Para recompor o plano amostral para  sua UF (que vai se chamar subamostra e que você vai utilizar no lugar de sample.pnadc)
%%R
subamostra <- svydesign(ids = ~UPA, strata = ~Estrato, weights = ~V1028, data = AP, na.rm=TRUE, nest = TRUE)

# para evitar erro "has only one PSU at stage 1"

options(survey.lonely.psu = "adjust")